In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Download Landsat NBR  and CCI Fire time series over sampled locations

This notebook downloads NBR time series (derived from Landsat Surface Reflectance data) and CCI Fire time series from sampled locations

The point collection is assumed to contain the field `first` (referring to the ecoregion).

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [2]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=mLrTMZb5P0Qi2E0QdU4-43zgY3ZjSDPZNAzSCjjvouc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7fH6qq1RMwiEIPFBj-Fx-yBVs-FVSkZNliGbuLKpiK8kXMSWA2OQQ

Successfully saved authorization token.


### User inputs

Enter the following inputs:
- `start_date`: start date of the period of interest
- `end_date`: end date of the period of interest
- `output_name`: name of the output file
- `points_collection`: name of the collection with sampled points
- `init_point`: initial point to be processed
- `end_point`: final point to be processed (setting this to `None` equals the last point in the collection)

In [7]:
# Inputs
start_date = ee.Date('1995-01-01')# start date of period of interest
end_date = ee.Date('2020-01-01')# end date of period of interest
output_name= 'samples_forest_fire_nat_n30.csv'#  path of output file
# name of GEE collection of points - from each of these points we want to extract a time series
points_collection='users/wandadekeersmaecker/RETURN/samples_forest_loss_fire_n30'
init_point = 0# start processing this point onwards
end_point = None# last point to be processed, if None: this equals the latest point



### List sampled points

Get a list of the sampled points 

In [8]:
# number of points in the collection
npontos=ee.FeatureCollection(points_collection).size().getInfo()

if end_point == None:
	end_point=npontos
print(end_point)
# GEE list of points
lista_pontos = ee.FeatureCollection(points_collection).toList(npontos)


1361


### Functions to prepare the Landsat Surface Reflectance data
The following functions are defined:
- `maskL8sr`: mask the Landsat 8 surface reflectance data 
- `maskL457sr`: mask the Landsat 4,5,7 surface reflectance data 
- `calcNBRl8`: calculate NBR for landsat 8
- `calcNBRl457`: calculate NBR for landsat 4,5,7

 

In [9]:
# function to mask Landsat 8 surface reflectance data
def maskL8sr(image):
	# Bits 3 and 5 are cloud shadow and cloud, respectively.
	cloudShadowBitMask = 1 << 3	
	cloudsBitMask = 1 << 5  
	snowBitMask = 1 << 4
	cloudconf1 = 1 << 6
	cloudconf2 = 1 << 7
	cirrusconf1 = 1 << 8
	cirrusconf2 = 1 << 9
	aeroconf1 = 1 << 6
	aeroconf2 = 1 << 7
	# Get the pixel QA band.
	qa = image.select('pixel_qa')
	aero = image.select('sr_aerosol')
	# Both flags should be set to zero, indicating clear conditions.
	mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0)).And(qa.bitwiseAnd(snowBitMask).eq(0))
	#Return the masked image, scaled to reflectance, without the QA bands.
	return (image.updateMask(mask)
		.select("B[0-9]*")
		.copyProperties(image, ["system:time_start"]))
 
 # function to mask Landsat 4, 5, 7 surface reflectance data
def maskL457sr(image):
	qa = image.select('pixel_qa')
	op = image.select('sr_atmos_opacity')
	# If the cloud bit (5) is set and the cloud confidence (7) is high
	# or the cloud shadow bit is set (3), then it's a bad pixel.
	# Bits 3 and 5 are cloud shadow and cloud, respectively.
	cloudShadowBitMask = 1 << 3
	cloudsBitMask = 1 << 5
	snowBitMask = 1 << 4
	cloudconf1 = 1 << 6
	cloudconf2 = 1 << 7
	
	mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
		.And(qa.bitwiseAnd(cloudsBitMask).eq(0))
		.And(qa.bitwiseAnd(snowBitMask).eq(0)))
	return (image.updateMask(mask)
		.select("B[0-9]*")
		.copyProperties(image, ["system:time_start"]))

# function to calculate NBR for landsat 8
def calcNBRl8(im):
	return (im.toFloat()
		.normalizedDifference(['B5', 'B7'])
		.rename('NBR')
		.copyProperties(im, ["system:time_start"]))

# function to calculate NBR for landsat 4,5,7
def calcNBRl457(im):
	return (im.toFloat()
		.normalizedDifference(['B4', 'B7'])
		.rename('NBR')
		.copyProperties(im, ["system:time_start"]))




## Extract time series
For each point, 
- Landsat time series are prepared:
 - import Landsat 4, 5, 7, and 8 surface reflectance collections 
 - filter on user defined time span
 - mask cloudy observations
 - calculate NBR
 - extract time series of point location
- Fire time series are prepared:
 - CCI Fire fire DOY and fire confidence are impored
 - filter on user defined time span
- a vector with the following information is added as one line to the output file:
 - `j`: ID of point
 - `ecoreg`: ecoregion 
 - `point.geometry().coordinates().getInfo()`: coordinate
 - `values`: Landsat image names
 - `anp`: Landsat observations
 - `freDts`: CCI fire (DOY) image names
 - `freVals`: CCI fire DOY observations
 - `freConfDts`: CCI fire (confidence) image names
 - `freConfVals`: CCI fire confidence observations





In [ ]:
# iterate over the points and write surface reflectance data to a file
with open(output_name,"a") as f:
	for j in range(init_point,end_point):
		print(j)
		# get one point 
		point = ee.Feature(lista_pontos.get(j))
		# Landsat 8,7,5,4 surface reflectance image collection, 
		#filtered using date range and spatial boundary, cloud masked, NBR derived
		colL8 = (ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
			.filterDate(start_date,end_date)
			.filterBounds(point.geometry())
			.map(maskL8sr)
			.map(calcNBRl8))
		colL7 = (ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
			.filterDate(start_date,end_date)
			.filterBounds(point.geometry())
			.map(maskL457sr)
			.map(calcNBRl457))
		colL5 = (ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
			.filterDate(start_date,end_date)
			.filterBounds(point.geometry())
			.map(maskL457sr)
			.map(calcNBRl457))
		colL4 = (ee.ImageCollection("LANDSAT/LT04/C01/T1_SR")
			.filterDate(start_date,end_date)
			.filterBounds(point.geometry())
			.map(maskL457sr)
			.map(calcNBRl457))
		# combine all collections
		col = colL8.merge(colL7).merge(colL5).merge(colL4)
		# convert the collection to an image where each band represents a different date
		original=col.select('NBR').toBands().reduceRegion('mean',point.geometry(),30)
		# Import the fire collection
		# collection containing the day of year of fire
		colFire = (ee.ImageCollection("users/wandadekeersmaecker/CCI_fire_v51").filterDate(start_date,end_date).filterBounds(point.geometry()))
		# collection containing the confidence of fire occurrence
		colFireConf = (ee.ImageCollection("users/wandadekeersmaecker/CCI_fire_v51_conf").filterDate(start_date,end_date).filterBounds(point.geometry()))
		# extract a fire time series for the point of interest
		fre = colFire.toBands().reduceRegion('mean',point.geometry(),30)
		freConf=colFireConf.toBands().reduceRegion('mean',point.geometry(),30)
		# get the fire information
		freList=fre.getInfo()
		freDts = list(freList.keys())# dates
		freVals=list(freList.values())# fire doy 
		freConfList=freConf.getInfo()# 
		freConfDts = list(freConfList.keys())# dates
		freConfVals=list(freConfList.values())# fire confidence
		# ecoregion of the point
		ecoreg = point.get('first').getInfo()
		a=original.getInfo()
		# extract dates of the observations
		values=list(a.keys())
		# extract observations
		anp=list(a.values())
		# vector to be exported
		write_vector=[j,ecoreg,point.geometry().coordinates().getInfo(),values,anp, freDts,freVals, freConfDts, freConfVals]
		f.write(str(write_vector)+'\n')
		f.flush()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
